In [5]:
import sys
sys.path.append('../')
import lib

In [13]:
import spacy
import json
import pandas as pd

In [40]:
example_path = '../data/articles_ICIBrazza_2020-02-01_2020-04-14.json'
with open(example_path, 'r') as reader:
    data = json.load(reader)
df = pd.DataFrame(columns=data['head'], 
                 data=data['articles'])
print(len(df))
df.head()

108


,id,title,text,Date,country,source,tweet
0,Congo_ICIBrazza_0,Congo : le gouvernement exhorte les parents à ...,L’humanité célèbre le 21 février de chaque ann...,2020-02-23T12:54:50,Congo,ICIBrazza,#Congo : le gouvernement exhorte les parents à...
1,Congo_ICIBrazza_1,Congo : le gouvernement exhorte les parents à ...,L’humanité célèbre le 21 février de chaque ann...,2020-02-23T12:54:50,Congo,ICIBrazza,#Congo : le gouvernement exhorte les parents à...
2,Congo_ICIBrazza_2,Un groupe de banques va racheter une partie de...,Le président Denis Sassou N’Guesso a reçu le 8...,2020-03-11T21:40:54,Congo,ICIBrazza,Un groupe de banques va racheter une partie de...
3,Congo_ICIBrazza_3,Congo : les braqueurs du convoi d’argent arrêt...,Les services départementaux de la police judic...,2020-03-08T11:00:30,Congo,ICIBrazza,#Congo - Les services départementaux de la pol...
4,Congo_ICIBrazza_4,Un groupe de banques va racheter une partie de...,Le président Denis Sassou N’Guesso a reçu le 8...,2020-03-11T21:40:54,Congo,ICIBrazza,Un groupe de banques va racheter une partie de...


In [46]:
clean_df = df.drop('id', axis=1).drop_duplicates(keep='first')

In [136]:
id_ = ['Congo_ICIBrazza_' + str(i) for i in range(len(clean_df))]
index = list(range(len(clean_df)))

In [137]:
clean_df['id'] = id_
clean_df.index = index

## We do not have that much info for a few countries (Congo, RDC, I believe CentrAfique is the same...

# What  do we do with the date ? The format ? Do we split it into year/month/day or sth like that?

In [142]:
#Original text (We have some \n, and potential other codes)
clean_df.iloc[-1].text

'©DR\n\nLe gouvernement congolais a annoncé une enveloppe de quatre milliards de Francs CFA au profit des ménages et personnes pauvres affectés par les mesures de confinement consécutives à la propagation du coronavirus.\n\nCette annonce a été faite ce vendredi par le ministre de la Communication, porte-parole du gouvernement, Thierry Lézin Moungalla à l’issue de la première réunion de la coordination nationale de gestion de la pandémie du Covid-19 qui s’est tenue sous l’autorité du président Sassou N’Guesso\n\nAinsi selon le porte-parole du gouvernement, il ressort de cette réunion plusieurs mesures économiques, sanitaires et sociales.\n\nAu plan social, le gouvernement a décidé du « transfert dans les prochains jours, de la somme de 4 milliards de francs CFA aux ménages et personnes pauvres déjà répertoriés dans différentes localités du pays par le ministère en charge des affaires sociales. »\n\nLa Coordination a réitéré la gratuité de l’eau et de l’électricité pour tous les ménages,

In [56]:
#Download spacy french vocabulary 
!python -m spacy download fr_core_news_md
#They also have multi-lingual stuff--> Can be interesting if we want to apply the method to other types of text

     |████████████████████████████████| 85.7MB 6.0MB/s eta 0:00:011
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.1.0-cp37-none-any.whl size=87463874 sha256=c74a4eb12a02d8fd2c47fa61c99d3baed1f9915c045c15a19e30e1d9adbf2aa9
  Stored in directory: /private/var/folders/gt/25j8ndxj1c52vv_v5bv0b_n80000gn/T/pip-ephem-wheel-cache-hzwud_06/wheels/7e/91/64/f61e597321455d6e42a76abac5736d919a265c31be451cc1ba
Successfully built fr-core-news-md
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [62]:
try:
    spacy.load('fr_core_news_md')
except Exception as e:
    print(e)
#That's the usual way to make it work... I don't know why it doesn't here
    

[E050] Can't find model 'fr_core_news_md'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.


In [61]:
import fr_core_news_md
nlp = fr_core_news_md.load()

In [139]:
text_ = clean_df.iloc[-1].text
text_ = text_.rstrip() 
text_ = text_.replace('\n', ' ')
text_ = text_.replace("  ", " ")
text_ = text_.replace('[^\w\s]','')
#lower_ = lambda x: " ".join(x.lower() for x in x.split())
#text = lower_(text)
doc = nlp(text_)
#get the "entities in the text"
doc.ents

(DR,
 Francs,
 ministre de la Communication,
 Thierry Lézin Moungalla,
 Covid-19,
 Sassou N’Guesso,
 CFA,
 La Coordination,
 Coordination nationale,
 Task,
 Covid-19,
 Trésor,
 Etat,
 Covid-19,
 Covid-19,
 Covid-19,
 Covid-19)

In [83]:
help(doc)

Help on Doc object:

class Doc(builtins.object)
 |  A sequence of Token objects. Access sentences and named entities, export
 |  annotations to numpy arrays, losslessly serialize to compressed binary
 |  strings. The `Doc` object holds an array of `TokenC` structs. The
 |  Python-level `Token` and `Span` objects are views of this array, i.e.
 |  they don't own the data themselves.
 |  
 |  EXAMPLE: Construction 1
 |      >>> doc = nlp(u'Some text')
 |  
 |      Construction 2
 |      >>> from spacy.tokens import Doc
 |      >>> doc = Doc(nlp.vocab, words=[u'hello', u'world', u'!'],
 |                    spaces=[True, False, False])
 |  
 |  DOCS: https://spacy.io/api/doc
 |  
 |  Methods defined here:
 |  
 |  __bytes__(...)
 |  
 |  __getitem__(...)
 |      Get a `Token` or `Span` object.
 |      
 |      i (int or tuple) The index of the token, or the slice of the document
 |          to get.
 |      RETURNS (Token or Span): The token at `doc[i]]`, or the span at
 |          `doc[sta

In [85]:
#print([t.text for t in doc]) 
doc.to_json()

{'text': 'L’humanité célèbre le 21 février de chaque année la Journée internationale de la langue maternelle. A cette occasion, le gouvernement de la République du Congo a prononcé une déclaration lue par le ministre de la Culture et des Arts, Dieudonné Moyongo.\n\nInstituée depuis 1999 par l’Organisation des Nations unies pour l’éducation, la science et la culture (Unesco), la célébration de cette année a eu pour thème : « Langues sans frontières ».\n\nLes langues maternelles et plus particulièrement les langues nationales qui sont les langues propres à un pays, constituent l’un des outils les plus performants et les plus structurants pour faire participer activement les populations au développement. Elles sont aussi un facteur déterminant pour la préservation de la diversité culturelle et un vecteur essentiel du patrimoine culturel immatériel, a indiqué le ministre Dieudonné Moyongo.\n\nPour la circonstance, le ministre de la Culture et des Arts a rappelé que l’usage des langues nati

In [140]:
text_

'©DR Le gouvernement congolais a annoncé une enveloppe de quatre milliards de Francs CFA au profit des ménages et personnes pauvres affectés par les mesures de confinement consécutives à la propagation du coronavirus. Cette annonce a été faite ce vendredi par le ministre de la Communication, porte-parole du gouvernement, Thierry Lézin Moungalla à l’issue de la première réunion de la coordination nationale de gestion de la pandémie du Covid-19 qui s’est tenue sous l’autorité du président Sassou N’Guesso Ainsi selon le porte-parole du gouvernement, il ressort de cette réunion plusieurs mesures économiques, sanitaires et sociales. Au plan social, le gouvernement a décidé du « transfert dans les prochains jours, de la somme de 4 milliards de francs CFA aux ménages et personnes pauvres déjà répertoriés dans différentes localités du pays par le ministère en charge des affaires sociales. » La Coordination a réitéré la gratuité de l’eau et de l’électricité pour tous les ménages, sur l’ensemble

# Things that could be interesting for the very first analysis:
- Count of tweets with time (probably more tweets/news since mid-march)
- Keywords with time
